In [13]:
import pandas as pd
import numpy as np
import sys
import tqdm
import os
import glob

In [14]:
try:
    import google.colab
    colab = True
    from google.colab import drive
    drive.mount('/content/drive')
    
    fine_tuned_filepath = '/content/drive/MyDrive/NLP_files/mybert.pickle'
    concepts_filepath = '/content/drive/MyDrive/NLP_files/concepts_strings_with_ids.csv'
    embeddings_output_filepath = '/content/drive/MyDrive/NLP_files/concepts_embeddings.csv'
    device_name = 'cuda'
    sys.path.append('/content/drive/MyDrive/NLP/bert_embeddings')
    print(sys.path[-1])
except ImportError:
    colab = False
    fine_tuned_filepath = '../models/mybert.pickle'
    concepts_filepath = '../data/concepts_strings_with_ids.csv'
    embeddings_output_filepath = '../results/embeddings/concepts_embeddings.csv'
    device_name = 'cpu'
from emb_helpers import * 


In [15]:
!pip install transformers

In [16]:
!rm -R /content/drive/MyDrive/NLP
!git clone https://github.com/hadaszm/NLP.git /content/drive/MyDrive/NLP

In [24]:
from transformers import AutoTokenizer, AutoModel

biobert = AutoModel.from_pretrained("dmis-lab/biobert-v1.1").to(device_name)

In [23]:
for file in glob.glob(os.path.join(os.path.dirname(concepts_filepath), os.path.basename(concepts_filepath).replace('.csv', '*.csv'))):
    print(file)
    output_filename = embeddings_output_filepath.replace('.csv',f'_{id_}_biobert.csv')
    if os.path.exists(output_filename): 
        continue
    concepts = pd.read_csv(file)
    concepts_emb = return_embeddings_for_concepts(concepts['best_string'], biobert)
    concepts_emb['concept_name'] = concepts['STR']
    concepts_emb[['concept_name'] + list(range(len(concepts_emb.columns)-1))]
    id_ = file.split('_')[-1].replace('.csv', '')
    concepts_emb.to_csv(output_filename, index=False)

../results/embeddings/concepts_embeddings_ids_biobert.csv
../results/embeddings/concepts_embeddings_0_biobert.csv
../results/embeddings/concepts_embeddings_1_biobert.csv
../results/embeddings/concepts_embeddings_10_biobert.csv
../results/embeddings/concepts_embeddings_2_biobert.csv
../results/embeddings/concepts_embeddings_3_biobert.csv
../results/embeddings/concepts_embeddings_4_biobert.csv
../results/embeddings/concepts_embeddings_5_biobert.csv
../results/embeddings/concepts_embeddings_6_biobert.csv
../results/embeddings/concepts_embeddings_7_biobert.csv
../results/embeddings/concepts_embeddings_8_biobert.csv
../results/embeddings/concepts_embeddings_9_biobert.csv
